In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:
from vexa import VexaAuth, VexaAPI
from chat import UnifiedChatManager, UnifiedContextProvider
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
from helpers import jupyter_stream_output
from uuid import UUID
from typing import Optional, List

In [4]:
auth = VexaAuth()
vexa = VexaAPI()
user_id = (await vexa.get_user_info())['id']

# Initialize search engines

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [5]:
import os
voyage_apikey = os.getenv('VOYAGE_API_KEY')

In [6]:
from chat import UnifiedContextProvider
from chat import UnifiedChatManager

In [7]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25


In [8]:
qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = ElasticsearchBM25()

Connecting to Elasticsearch at elasticsearch:9200


In [9]:
context_provider = UnifiedContextProvider(qdrant_engine=qdrant_engine,es_engine=es_engine)

In [10]:
from IPython.display import Markdown

In [19]:
display(Markdown(await context_provider.get_context(user_id=user_id,query="what happened last week?")))

## meeting_id 8 - November 25, 2024 14:00

- [00:01] The chunk is part of a conversation where the speaker is reflecting on a situation, possibly expressing uncertainty about their own actions or decisions related to a previous topic discussed, indicating a moment of introspection or self-doubt.
  > :  Может быть, я ее сам...
- [00:00] The document appears to be a conversation where the speaker is discussing a potential issue, suggesting that something is skipping or malfunctioning, and speculating on the reasons behind it.
  > :  Так, ну оно скипит, наверное, по какой-то другой причине.

## meeting_id 23 - November 20, 2024 16:40

- [42:12] This chunk is part of a technical discussion between Artem Puzik and Dmitriy Grankin regarding code changes and debugging in a software development context. They are addressing issues related to version control, specifically focusing on tracking changes and resolving potential conflicts in their codebase.
  > Artem Puzik:  А ты попробуй сейчас что-нибудь двинуть.  Почему-то только мои изменения, действительно.  Я сейчас все откатил.  такое компонент 4 и в какая строчка у меня 32 33 у тебя как бы если они есть то будут до 32 33 32, 33, у меня 32 пустая, 33, handle, mouse, down.  Да, то есть смотри, то, что я сейчас внес, оно...  Наверное, в этом дело.  Сейчас я пробую рефреш делать.  Да, давай я сейчас уйду отсюда, потому что иначе у нас конфликт может быть.  Типа там...  Так, так, я тогда...  Делать рефреш.  Мне важно, чтобы гид понял, что я что-то поменял.
- [22:01] The chunk is part of a conversation between Dmitriy Grankin and Artem Puzik discussing the functionality of a chat system, specifically focusing on how threads are managed and referenced within the chat application. They are trying to understand how to continue a previous chat and the behavior of threads in the system.
  > Dmitriy Grankin:  Нет, у меня же где-то уже...  А Threads вообще все Threads должен спрашивать, да?  Ну, в теории-то, нет, я же видел...  что у меня два треда где-то бегает.  Я же спросил hello и так далее.  То есть мы сейчас хотим продолжить чат с предыдущего.  Или здесь.  Хорошо.
- [66:39] The chunk is part of a technical discussion between Dmitriy Grankin and Artem Puzik, focusing on debugging and improving a chat application. They are addressing issues related to message handling, thread management, and the functionality of the chat system, specifically questioning the effectiveness of certain features and the importance of context in conversations.
  > Dmitriy Grankin:  не знаю например что не такое чтобы было очевидно вот фиш не работает не ну допустим это получается что да нам надо в инстанс давай я пока проверю что это точно не смотри подожди самый  А я... Hello, role, content, hello...  слава пусть мы спикер вот смотри я не спросил кто более важный
- [10:21] The chunk is part of a conversation between Dmitriy Grankin and Artem Puzik discussing the functionality of a chat application, specifically focusing on how messages should appear in the sidebar after a response is received.
  > Dmitriy Grankin:  Вот справа в сайт-баре у тебя после того, как он ответит, у тебя должен появиться...  этот срет должен появиться в сайт-баре.  В сайт-баре? Related messages, да?  Нет? Нет, слева в сайт-баре у тебя должен  он был появиться. Он там где-то есть, он что-то
- [00:41] The chunk occurs during a conversation between Dmitriy Grankin and Artem Puzik, where they discuss various topics related to their work, including language learning and technical issues in a project. Artem expresses surprise about a potential eavesdropping situation, questioning whether it was explicitly mentioned or if it was a glitch.
  > Artem Puzik:  Ничего себе тут прослушка. Я удивился.  Так и сказали? Или это глюк?

## meeting_id 6 - November 25, 2024 13:11

- [00:34] The chunk is part of a conversation where Dmitriy Grankin is expressing confusion or curiosity about the current situation or events occurring during a discussion or meeting.
  > Dmitriy Grankin:  Что такое происходит?
- [00:47] The document consists of a conversation involving Dmitriy Grankin, who is discussing the status of a project or system, expressing uncertainty about the current developments and the visibility of new additions. The chunk reflects his observation about not seeing any updates or changes.
  > Dmitriy Grankin:  Мы пока не видим, что у нас добавились какие-то...
- [00:40] The chunk occurs during a conversation where Dmitriy Grankin is discussing the status of a project or task, expressing confusion and seeking clarity about the situation.
  > Dmitriy Grankin:  Так.
- [00:32] The document consists of a conversation involving Dmitriy Grankin, who is inquiring about the current status or developments in a situation, expressing curiosity and confusion about what is happening.
  > Dmitriy Grankin:  И что там такое у нас происходит?
- [00:00] The document consists of a conversation involving Dmitriy Grankin and other participants discussing the status of a project or system, expressing confusion and seeking clarity about ongoing developments. The chunk reflects the initial action of starting up and observing the situation.
  > Dmitriy Grankin:  Запускаемся и смотрим, что там такое происходит.

## meeting_id 1 - December 02, 2024 12:10

- [00:00] The chunk is part of a conversation between Tatiana Sukhova and Dmitriy Grankin, where they discuss their recent social experiences and emotional states, specifically focusing on Tatiana's feelings after meeting new people over the weekend.
  > :  Привет, Таня.  Привет.  Как настроение?  Привет. Как настроение? Огонь, настроение огонь. Класс. Вчера что-то было не очень, я поняла, что я об людей подразрядилась за выходные. Много встретила новых людей, которых не знала до этого, и меня это немножко подразрядило.
- [00:37] The chunk is part of a conversation between Tatiana Sukhova and Dmitriy Grankin, where Tatiana shares her experiences from a busy weekend filled with social activities, including poker, tennis, and dining with friends, reflecting on the impact of these interactions on her energy levels.
  > Tatiana Sukhova:  что за что что за тусовка 3 тусовки подряд пятница суббота воскресенье У нас был покер, как обычно, раз в две недели Вадик проводит.  В субботу мы друзей пригласили в теннис поиграть, но что-то как-то пришло их больше, чем мы ожидали.  И потом мы, конечно же...  Зашли на бокальчик, и это продлилось до 11 часов вечера.  То есть мы в теннис большой поиграли, потом у нас настольный теннис, потом мы ели устрицы.  В общем, о воскресенье мы на паузу.  Падал, как обычно, фамилия падала.  Там тоже были все новенькие, вообще почти не было стареньких.  А так все нормально.  Хорошо, что это во время сна компенсируется.

## meeting_id 31 - December 06, 2024 12:05

- [09:50] This chunk occurs during a discussion about the business model and user metrics of a transcription and meeting summarization tool, where Dmitriy Grankin inquires about the subscription model, user count, and retention rates, following a broader conversation on market competition and differentiation strategies.
  > Dmitriy Grankin:  like a purchase acquisition and there is no way to go but it will be as i see the market at least two three four years So, how does it work? Do you have enterprise B2B subscription model or how does it work?  For now, it's just free of charge. Just do what you want.  And so do you count the number of users you have so far?  Of course, yes, yes, yes.  So how is it looking?  It's like 300, 400 users.  about those who are using at least three meetings a week, there is about 50 of them.  And the group retention, they are marked Weeks and months during like some other 500 meetings they have recorded with the crew.

## meeting_id 38 - November 28, 2024 13:12

- [03:29] The chunk occurs during a conversation where Dmitriy Grankin expresses frustration and confusion about a situation involving a new database, indicating a tense and chaotic atmosphere.
  > Dmitriy Grankin:  Don't.
- [05:07] The chunk occurs during a conversation where Dmitriy Grankin expresses frustration and confusion about a situation involving a new database, indicating a tense and chaotic atmosphere.
  > Dmitriy Grankin:  Nieeeet.
- [05:26] The chunk occurs during a conversation where Dmitriy Grankin expresses frustration and confusion about a situation involving a new database, indicating a breakdown in communication or understanding with another person.
  > Dmitriy Grankin:  Ты что, с ума сошел, дебил?

## meeting_id 9 - December 17, 2024 04:29

- [06:40] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing their experiences with HubSpot, including Dianna's reflections on her husband's past income, changes in farming, and updates related to their HubSpot partner business.
  > Dianna Lloyd:  I think what Andrew said, He, when his, when we first got married, he was making 40,000 a year and supporting the two times the times have changed, hasn't it?  Farms have changed it.  I saw just now, I just logged into HubSpot and I can see people and things.  Okay.  I don't know what happened because I did it last week, but I must have not saved it.  Or I tried to copy a permission set and it didn't work. But yes, I just, after looking at your email, I was like, oh, I'm pretty sure I did it.  So I've updated it now.  I won't make a comment about like, you know, that I own a partner business for HubSpot.  I own a partner business.  I'm a HubSpot partner.  I'm not a partner.  You own a partner business.  No, I would not say that.  That's what this is.  You're a HubSpot partner business, agency.  Yeah.  Or you do digital marketing that does part HubSpot.
- [25:07] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing the use of documentation and communication tools for managing client interactions and project workflows. They explore the effectiveness of Google Docs versus Slack for tracking questions and answers, the importance of understanding HubSpot's lead object, and the need for better organization in their work processes.
  > Dianna Lloyd:  Um, let me think about that. I think you could set up like a notification or something like that. But, but I just the idea, it doesn't have to necessarily be a Google Doc. But I mean, as soon as you tag somebody off.  Obviously, that works.  Yeah, yeah, that's true.  But let me just have a look and see if I can find it.  We can talk through it and see if there's a solution that works for us, because I was just thinking, you know, this is going to get bigger than just you managing, right?  And so you're going to need to see what I do and what, and I think just looking at my tasks is not always going to show you what the questions have been and what has worked on yeah you know and i and i think i think it's probably a good idea i've never had to work with a document like that other than with the hubspot csm yeah but it was good it was really good i found it good i found it helpful i was always using it we were back and forth in the con you know and she would go out and get answers and come back to it when she wanted you know what i mean but it was like this little bulletin board for the two of us like a slack channel i guess in a way but slack doesn't hold on to data long and that's the problem with slack is your your account i think clears after 90 days you lose you lose information nation no not not not ours for me it said it it only holds 90 days of history  does it still say that now because it Shouldn't, because I'm paying for you as a user.  It said that to me before.  I'll check.  So are you sure you're...  Yeah, I've got all...  All time?  Yeah.  I've got all my yeah all time. Yep. Okay, I got a message that mine was 90 days  So I'll just double check that that was a couple of days ago. I thought that was last week. I Was looking for something and it said 90 days maybe it's a setting you've got for me no it's that i need to upgrade  no you're a regular member so you're you're a paying member That shouldn't be the case, but...  Oh, look, okay.  Yeah.  But it's probably not the best way of keeping your history of everything anyway.  Yeah.  No, yeah, because everything is – yeah, I'm paying for it, so it should be there.  Oh, one – one – I mean, we could –  That one is something like this in Slack, but they have to be on Slack.  And I think Google's might be better because yeah, there's like channel and all of that and Slack, you're going to get all over the place.  And it's nice to have like just like I've got one client you know what i mean yes yeah so um okay and then i've got a 430 and then um We can talk about.  I've gone through it, but like at some point I'd like to love to have a coffee and just sit down and like, or a beer or something like that and have a chat about some thoughts I did.  Like I said, last week.  spent a lot of time learning about the the lead object um and how it might be used and i think i have a much better understanding of using it moving forward i think it's not I think it's something that people are kind of ignoring.  But I think HubSpot are going to be pushing that it gets used.  Yeah.  Well, it's relatively new information.  Yeah, it's new. It's been the last six months, I think, or a year.  It's only, like, it came out, I think, like, April or May this year, so...  Anyway, I went through a couple of videos and watched and I got like, you know that one of them was really talking, I grabbed this from one of the videos and I was like, Oh, this is really cool.  And it was, you know, talking about the different life cycle stages and how they saw it mapping and how they work together.  Yeah.  And so I thought that was a useful...  way of to help somebody understand how these different things work together because i'm sure that clients probably really don't understand that no they don't  I find it hard.  I actually think that they, and then you've got like this life cycle stages, and then you've got contact life cycle stage, and then company life cycle stage, which I'm assuming they'd have to be absolutely  completely married yeah they're synced yeah so they're synced on the back end so  okay anyway that's in there so whenever you want to go look at it um So, yeah, so still doing the HubSpot Academy stuff.  If you have anything that you would like me to do, I've started with this one.  Yeah, awesome.  Love it.  It's a big, chunky one.  Yeah.  That's a good one, yeah.  Have you done it?  I think I've done it at a previous – when I was working for a first job.  Pretty sure I did the certification, yeah.  Okay.  done it in real life yeah same yeah and this is the thing it's like the certifications are really
- [40:12] This chunk is part of a conversation between Marcus Torrisi and Dianna Lloyd discussing the implementation of a meeting link system in HubSpot for managing client interactions. It details the workflow process for handling leads, including how to enroll contacts in sequences, manage responses, and communicate with the OLM team regarding lead status. The discussion highlights the integration of HubSpot with third-party systems and the challenges faced in tracking lead interactions.
  > Marcus Torrisi:  I've just created a meeting link in HubSpot and then on that, there's a form they submit that form and select the time and then the treatment coordinator now has that in their calendar to call them yeah okay whose meeting link is it it's it's the clinics have a generic one so they have like a sydney at next mile okay and that and that calendar everyone works off so then yes that's their shared inbox shared calendar yeah it's something like that but they use it one calendar which then it gets put in their calendar at that time they give them a call that was a good solution well i i would love to i don't think we're gonna have enough time but i will run you through it um because i know you have a call at four 4 30. yeah let me let me really quickly run it through okay because i think it it's a it's i presented presented it to jody on last thursday and she was happy with it so it's a good game  It's a, it's a really good solution.  It's actually much more simple than what we were saying.  What we were saying was basically just a form on a landing page that they go to.  It's it's six of one almost in a way, because you can just create a form in a fake little landing page that only they got to and they just fill it in right it's it's the same sort of thing in a way can you see my screen yeah okay so there's  one two three and four workflows so this is the first email so after they go through the main workflow, so they get assigned to the clinic, they then go into this workflow here, which then sends them all the details.  yep all right and that sends to an email address that since yeah because they haven't accepted  their invite so then after two days so that means all of them gets the lead and they're contacting that person yep after two days they and we delay until monday tuesday wednesday or thursday or friday the reason is that obviously we don't want it to run on yet the weekend we delay until 9 15 and then we enroll them in the sequence because they have sales enterprise sales enterprise they can do this you can't do this on pro  do you do if it wasn't there would have to be another massive work around  you have to do it off form submission yeah that's a whole other conversation so They're rolled into the sequence.  Now, what then happens is, is that OLM is contacting that person, contacting the person.  On day three of that person submitting the form, they get that first sequenced email.  Wait, can I ask you a question?  The question.  Yep.  I can trigger a sequence off a list, right?  No.  Oh yeah.  If you, or if it's enter, if it's enterprise, the only on professional, you can only auto enroll.  someone for two things one they fill out a form on a website or two they like they have an interaction with your website it's like a page view or something like that okay But you can't auto-enroll them off a particular criteria.  Oh, okay.  Like a workaround could be is like you create like some sort of zap or something.  That if they meet a certain criteria, it zaps them like they filled out a form, and then they would enroll in that workflow.  Yeah, I've done similar things like that.  So, okay.  Yeah, I understand what you're saying.  Yeah, it's like a bit of a workaround.  But anyway.  Okay.  Now, so what happens now is that obviously OBLM has those meeting links on hand.  So you've got the request to call back for next mile Melbourne.  and you've got the request to call back for Perth.  Perth is now included in this.  So what now happens is that if the person gets in touch olm follow them get in touch with the person and they want to book in they fill out one of these forms using my annotation at that point it sets them as an sql yeah Workflow number three, if they are in SQL, we unenroll them from the sequence that they're in, which means that they don't need to be contacted anymore.  been is because HubSpot doesn't know that that person has been being contacted because it's not it's not sitting in HubSpot it's sitting in OLM's stupid third-party system I don't even know why we use  Why wouldn't you add some sort of custom property just for historical reasoning, saying OLM contacted them or whatever?  Yeah, well, well, I'm in the workflows before in this first one.  I think I'm going to add something like tagged as OLM, like OLM, like worked OLM lead or something like that is yes, that we can segment it down the line.  Yeah, and it's also like they've approved calling and all of that kind of stuff, right?  It's not called calling.  That's right.  Okay.  Exactly.  Then we've got this workflow here, which goes, this workflow basically says like they've replied to an email, a sales email and they're in a sequence.  So what that then does is obviously, that's going to automatically unenroll them from the sequence but we then we need to notify OLM that that person has replied to the sequence and the treatment  is now handling it but if the sequence is sitting sorry if the sequence is sitting with um smile yeah why are you letting because olm will follow up if smile doesn't get a hold of them or  something no because what happens is is if like someone because they're enrolled in a sequence if they reply to that sequence OLM doesn't know that you cut out just I've lost you Sorry, my little rural dongle died.  I've got to charge it.  Sorry, but I'm back on my phone, so it's okay.  That's okay.  So what, I can't remember where I was, but this worked for a basic, because what happens is the contact's enrolled in the sequence, but OLM doesn't have, of access to HubSpot, so if they reply to that sequence saying, yes, give me a call, the treatment coordinator gets that communication.  So what we need to do, like this notification here basically emails olm and goes they've replied to a sequence don't contact them anymore because now it's on the treatment coordinator to handle it so it's kind of acknowledging the little hole that you've got there  and Yeah, right.  Just trying to cover for that.  Okay.  The gap, yeah, because they don't know.  So, yeah, that's important.
- [01:11] This chunk is a conversation between Marcus Torrisi and Dianna Lloyd, where Marcus shares his experience of being in rural New South Wales, discussing the property he is staying at, his plans for the week, and mentioning a trip to Junee, known for its licorice factory.
  > Marcus Torrisi:  what's up oh nothing much i'm in uh beautiful rural new south wales and it's stinking hot i'll send you actually i'll send you some photos oh yeah are you on a property We're on a large property, but we're not on a farm.  The property that we're on is quite big, but they don't live on a farm.  have like all the other family and stuff are on the farms let me um just the commune  yeah let me it's such a nice house that they um  This is your girlfriend's parents.  Sorry, say that again.  This is your girlfriend's parents.  Grandma and auntie live here.  So.  it's um yeah it's really a nice day like i'm working did a little bit of work yesterday morning and then working kind of today i'll probably log off probably after this call a little bit Wednesday, we're going to Junee.  I don't know if you've heard of that.  Yeah, where the licorice factory is and where a bunch of other families are.  live which is gonna be cool and then uh friday probably do a little bit of thursday and friday do a little bit of work those days and then yeah come back i just i just sent through those photos have a look pretty uh where did you send them

## meeting_id 28 - November 25, 2024 15:20

- [01:04] The chunk occurs during a chaotic discussion where Dmitriy Grankin expresses confusion and frustration about ongoing updates and issues related to a registration process, reflecting a sense of urgency and disarray in the situation.
  > Dmitriy Grankin:  Шло время обновляться.  Obrigado.  Dicilo prima.  Что происходит? Я ничего не понимаю, что происходит.

## meeting_id 33 - December 04, 2024 15:32

- [25:15] This chunk is part of a conversation between Jasper Vanuytrecht and Dmitry Grankin, where Jasper discusses the importance of outreach messages and use cases in sales strategy. He offers his consulting services to help business owners develop effective sales strategies, emphasizing the need for personalized support and implementation guidance. The discussion also touches on the costs associated with his services and the challenges of balancing product development with sales efforts.
  > Jasper Vanuytrecht:  That's, I think, important to mention also in every conversation with such a client, you know, and so in the outreach messages, So, yeah, I think Yeah, and then  Sorry, yeah, just dropped Then comes obviously comes use cases And the slot of use cases use cases which we don't have a good understanding which like marketing research like you can can make it like one from marketing research who are competitors what people say about this company about this product performance All of our team members are more into this and that technology, this and that idea.  about crypto or whatever yeah yeah well if  again like if you're planning to do the sales yourself you're not going to hire like a lead generation agency you're not going to like hire salespeople like it's very expensive like you know the salaries you know sales commission or especially the lead generation agencies they charge maybe every month for okay so if you want to have like better like a strategy that you can do the sales but in a very effective way, I can help you with that. And my offer is very, it's very simple. So I always work with business owners, like one session per week of more or less two hours we do four to five sessions so it's like more or less one month and with all this you You will get the strategy and you know exactly which tools you can use to generate new clients, you know.  And during all this time, I'm like your, support you know um you can always ask me questions i will help it's not just like sessions or workshop sessions but i can help you on the go you have we have like collaborative documents uh we have the slides we have yeah help with the implementation execution of everything so all this together um to give you an idea of what it what it might cost if you're if you're interested um it ranges uh so the various the minimum it's like 2k the maximum is 5k so the bigger companies they more or less but about 5k they usually need more sessions so it's The amount of session that defines it, but you can, you can do it for.  For the lower ends of the, of the range to, to give you an idea.  I tried the same thing where I had one session with a friend of mine about this was the same concept. It didn't work just because I just didn't have time for implementation, just didn't have resources. Basically that Yeah, that's it, and I went building the product.  Yeah.  So...  it might happen is that I feel like at some point in time, if I feel like I'm dedicated to it right now for, Yeah, or a week or two and then go for it. I need My guidance say but it would not work on a monthly basis.  phases obviously because i need to switch back and forth between things
- [05:42] This chunk is part of a conversation between Dmitry Grankin and Jasper Vanuytrecht discussing a product related to meeting summaries and knowledge management. They explore features of the product, its user base, funding status, and client acquisition strategies, highlighting the importance of outreach and marketing efforts for growth.
  > Dmitry Grankin:  And after the meeting we got a dashboard like like this one which gives a summary and it gives a chat gpt style chat which can generate stuff based on a specific meeting or basically all the meetings as well.  What happened last week?  Tactic AI extension.  Tactic?  Yeah, I've been playing around with this one because it's a pretty similar concept.  uh-huh but it's putting this in Russian for some reason probably because I've got a lot of context in Russian no yeah Yeah, so the idea is basically to create knowledge that's accessible in in charge of the key manner.  Okay.  And it's it's success.  both real time during the call and after the call yeah so it's it's more like  like a chat interface, no, instead of static reports, you can.  The idea is that like chat is, is, is kind of central thing.  But also the meetings and the filtering is that I can filter a subset of meetings and chart over this specific subset.  Like in a future release, it will be like you could just filter out your clients or team members.  numbers and chat over these ones all right kind of free trial  for the product yeah yeah you can you can go and uh  and use it full like a full version of it for free now because like payment is not implement  the theme is not implemented yet this feature is not yet delivered well it's a good way to to get traction, to get some users.  Yeah.  To see how they perform.  Like it's in the pipeline,  but for now it looks like it is a product that's ready to to get some sales because uh there is a stable user base like with some people are using the product since early days for 500 meetings a few hundred victims there's a few users like this and we got a stable influx of users every day like a few like two to five users daily with no marketing whatsoever yeah yeah I think it's a solution for companies that have  of meetings and several people in the company, you know.  So do you have your VC backed or was it the previous group?  yeah it's VC backed like I've raised some angel investment mm-hmm Now this year Okay, and I think we'll be able probably be able to to go this way up to serious a maybe all right you said you have clients like new clients they go from your existing  network community like all warm no not really for now these are the clients that are coming with  empty utms which which basically means it's a direct uh customer directly referenced by someone i oh yeah yeah referenced right yes i don't i don't see the reference itself because oh yeah yeah referenced right yes i don't i don't see the reference itself because  but you're not doing like uh outreach to businesses to offer the trial or to talk to them about yeah how they are doing their meetings currently you know they're sharing
- [01:20] The chunk occurs during a conversation between Jasper Vanuytrecht and Dmitry Grankin, where they discuss their backgrounds and reasons for living in their current locations. Jasper inquires about Dmitry's move from Russia, leading to a brief exchange about Dmitry's escape in 2021.
  > Jasper Vanuytrecht:  I live here for four years already.  Did you escape Russia?  Earlier.  Oh, okay.  In 2021.  Not because of this.  Yes.
- [00:00] This chunk is part of a conversation between Jasper Vanuytrecht and Dmitry Grankin, where they discuss their current locations and the weather while Jasper is on vacation in Catalonia and Dmitry is in Cascais, Portugal.
  > Jasper Vanuytrecht:  how is it going good I'm spending some days at the coast here in Catalonia  what's the weather like it's a little bit cold uh like well  cold you know 10 15 degrees oh yeah but it's funny and you're in cascades uh yeah not far  It's about 20, I think now, this morning it was 16.  Okay, and it's Cascais, right?  It's 20 kilometers from Kashkash.  Okay.  If you're familiar with the area, I can be more specific.  Okay.  I've just been to Lisbon yeah it's it's about the same distance from Lisbon to like 20 kilometers

## meeting_id 39 - November 25, 2024 21:06

- [00:33] The chunk is part of a conversation where Dmitriy Grankin is testing a system or process, expressing curiosity about its functionality and results, and indicating a specific time reference (21.07) related to the test.
  > Dmitriy Grankin:  посмотреть как это у меня будет срабатывать что-нибудь как-нибудь вот мы сейчас почистим посмотрим как такая штука получается 21.07, да?

## meeting_id 40 - December 02, 2024 17:27

- [30:36] This chunk occurs towards the end of a conversation where Dmitriy Grankin is reviewing a tool called Tactic, discussing features such as labeling and action items, and expressing confusion about certain functionalities while transitioning to the next topic.
  > Dmitriy Grankin:  label conference long create new label labels label yeah labeling is a good idea okay basically that's it okay let's move forward action okay because me i told so which i just don't get it why what's that maybe it's um kann da gast und prompt automatically insert um ok, my prompt Herzlich Willkommen bei Tchau.

## meeting_id 30 - November 24, 2024 22:09

- [00:00] The chunk is part of a conversation where Dmitriy Grankin is discussing technical issues and troubleshooting related to a session or project, expressing confusion and seeking to understand the current situation.
  > Dmitriy Grankin:  Пробуем еще одно делать.  Что у нас такого происходит?

## meeting_id 24 - November 25, 2024 13:30

- [00:00] The chunk expresses strong negative emotions and insults directed at an unspecified individual, reflecting frustration or disappointment, and is part of a larger dialogue that continues with similar sentiments.
  > :  Идиот просто какой-то, кретинический.  Кретин просто.

## meeting_id 25 - November 25, 2024 20:17

- [00:00] The chunk is an opening remark in a conversation where participants are discussing the status of a project or task, indicating a positive development as someone has arrived or joined the discussion.
  > :  Пошла пляска.  Вот он здесь появился, это хорошо.
- [00:41] The chunk is part of a conversation involving Dmitriy Grankin discussing the current status of various tasks and processes, indicating a relaxed atmosphere as certain activities and individuals are on hold.
  > Dmitriy Grankin:  появилась никакого как у транскрипта Это нормально. Пока нормально. Мультирасист пока отдыхает.  Мальчик соседа тоже пока отдыхает.  Также прочая макаронная фабрика тоже отдыхает.
- [01:26] The chunk occurs during a discussion about the functionality and issues related to a system or process, where Dmitriy Grankin expresses frustration over updates and potential resets, referencing a problematic individual responsible for closing sessions.
  > Dmitriy Grankin:  смотри-ка все-таки обновляет и все сбрасываем Или нет? Или да? Или нет?  Тот мерзкий ублюдок, который закрывает.

## meeting_id 5 - December 11, 2024 18:31

- [00:38] The chunk occurs early in a conversation between Dmitriy Grankin and Lara Vargas, where they discuss unexpected events in their personal lives before transitioning into a professional discussion about influencer marketing strategies and outreach methods.
  > Dmitriy Grankin:  Today I had unexpected events about kids.  It's okay, sorry about that.  So last week you had asked me to think about ways to make my connection, is my connection good?
- [00:00] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin, where Lara discusses a recent incident involving windows falling from a building, expressing concern about the construction quality. The dialogue transitions into a discussion about marketing strategies and influencer outreach.
  > Lara Vargas:  the code uh that's what the the fire brigade said maybe the code made the glasses uh change the the you know the the structure and so they fell on the street it was it was a bit scary but thankfully everything is settled now they had to come in and and change everything it's pretty strange that windows just fall out of the building  yes apparently it had already happened to  other people before in this in the building oh yeah yeah okay so so i think it's it's uh the way that it was built it was bad yeah i don't know some mistakes in the construction or yeah yeah anyway that yeah i also had
- [09:22] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin, where Lara discusses ideas for utilizing the VEXA tool in innovative ways beyond its traditional use. She specifically focuses on applications for coaches to enhance communication feedback and outlines potential social media strategies for promoting these ideas.
  > Lara Vargas:  Yeah, so from what I gathered from our conversation last week, the briefing was think about ways to think to make VEXA a useful tool outside the...  usual way of use, think about tips and tricks to use it, to make content around it, and make research and About what type of different uses that would be.  So I brought a few that I thought about.  And so the first one would be a training Feedback for communication, so for example, people that are coaches, career coaches, you know, that they can use VEXA for their clients.  when they are in coaching their clients really, and then they can use a transcription to analyze how the person communicates during the meetings, look for, patterns and their communication as, for example, if they use some kind of excessive terms or if they interrupt sometimes how efficient the message is that they are transcribing and then sorry okay okay yeah go ahead and then add the social media that i would focus for this specific The specific target would be LinkedIn and Instagram.  The strategy would be short videos, visual testimonials and carousel.
- [12:39] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin discussing the potential applications of VEXA, a transcription tool, in various business contexts. Specifically, Lara is explaining how VEXA could be utilized in qualitative research, particularly in focus groups for product launches, highlighting its usefulness in analyzing participant feedback.
  > Lara Vargas:  So for example, if there is a big, a large company that wants to launch a product, and so they will make a focus group about that product you know yes so they will ask that focus group what they think about that product that normally happens with a record regarding that they make an interview with the person about what they think of the product.  So Vaxxer could be a really good use.  It's the kind of product
- [01:35] This chunk occurs early in a conversation between Dmitriy Grankin and Lara Vargas, where they discuss technical issues with their video connection before transitioning into a discussion about influencer marketing strategies.
  > Dmitriy Grankin:  unstable is it yeah i can hear you well i can see um some lagging in the video in the video i'm not sure why is it the connection or it's performance issue yeah but it's okay okay who's gonna start Yeah, I've got I've been exploring some ideas about this influence marketing Okay, so you can go ahead because my  hey my internet's a bit bad so i would rather let you start

## meeting_id 4 - December 05, 2024 10:02

- [05:05] This chunk is part of a conversation between Dmitry Grankin and Oleg Ageychev discussing the capabilities of a knowledge assistant for Google Meet, specifically how it can respond to queries based on past conversations and accumulated information.
  > Dmitry Grankin:  тему через просто через чат контексте этого вот но и могу собственно вот такую штуку но могу сказать what happened last week что-нибудь такое и он ответит в контексте сейчас  все беседы или вообще всего что он знает всего что он насобирал потому что вот
- [38:05] The chunk occurs at the end of a conversation between Oleg Ageychev and Dmitry Grankin, where they discuss the progress of Dmitry's project, Vexia, and the need for competitor analysis. Oleg suggests concluding the discussion and mentions he will look into the competitors while agreeing to stay in touch.
  > :  хорошо дим на этом давай наверно закончим на сегодня ссылки на конкурентов ну из  Вексу я сейчас тоже буду крутить, и что-то с ней пока поваляется.  Окей, договорились. Давай.  Давай, на связи.  Пока.

## meeting_id 17 - November 24, 2024 13:00

- [01:18] The chunk expresses a feeling of confusion or surprise regarding the speed of a previous discussion about connectivity checks, which alternates between working and not working.
  > :  Мне это слишком-то быстро как-то верно.

## meeting_id 13 - November 25, 2024 20:50

- [01:23] The chunk is part of a conversation where Dmitriy Grankin expresses his exhaustion and frustration after a long day, while discussing the status of a task or project that is expected to be completed soon.
  > Dmitriy Grankin:  Короче, в очередной раз умотался, ушатался.
- [02:48] The chunk is part of a conversation where Dmitriy Grankin discusses the status of a task or project, expressing fatigue and the absence of packages, leading to the decision to close the matter.
  > :  Теперь мы её, например, закрываем.
- [00:13] The chunk is part of a conversation where Dmitriy Grankin discusses the status of a task or project, expressing fatigue and the expectation that it will soon be completed.
  > Dmitriy Grankin:  И, по идее, эта штука должна сейчас у нас закрыться.

## meeting_id 36 - December 13, 2024 11:30

- [24:23] The chunk occurs towards the end of a conversation between Viktoria Kharlamova and Dmitry Grankin, where they discuss plans for the upcoming quarters, client load, and the timeline for potential collaboration on product onboarding and monetization strategies.
  > Viktoria Kharlamova:  Смотри, у меня вопрос по поводу того, я сейчас в финале планы на Ку-1.  На Q2 у меня, ну, первый квартал и второй квартал именно по клиентам и по загруженности.  Вот, у меня осталось два местечка, и на следующей неделе еще примерно пять звонков.  Вот, и, соответственно, я хотела бы понять, сколько тебе времени нужно подумать, принять решение и с точки зрения начала, если бы ты хотела начинать, когда бы это было?  Слушай, ну ставка такая для меня драконовская довольно-таки.  Да, чуть попозже.  Ага, окей, окей, я поняла.  Ну смотри, тогда в любом случае, я думаю, что...
- [00:09] This chunk is part of a conversation between Dmitry Grankin and Viktoria Kharlamova, where Dmitry discusses the development and features of a product he has been using, referencing a previous interaction in August and indicating that there have been updates or changes since then.
  > Dmitry Grankin:  полностью все пересобралось.  Но в основном это разработка.  Я тебе покажу сейчас, как продукт выглядит.  А я им пользуюсь, кстати.  вами в августе по моему созванивались вот тогда я себе скачал и какое-то время попробовала вот но сейчас может тут другой уже ну да в общем здесь с точки зрения расширения  Во время встречи у нас, наверное, не сильно что-то изменилось. А во время встречи у нас получается интересный продукт.

## meeting_id 35 - December 16, 2024 19:32

- [08:00] This chunk occurs during a conversation between Isabelle Santos and Thiago Arraes, where they discuss the progress of a project, including testing and communication within a group. Isabelle suggests sending a message in the group and inquires about the timing for testing, indicating a collaborative effort to resolve issues related to their work.
  > Isabelle santos:  lá no grupo.  Certo.  Você está lá também, né?  Estou.  Pode mandar um mensagem logo.  And I'll say it now, we'll look it up.  Peace.  Eu vou marcar a dúvida, que eu acho que a gente vai mais à frente.  Aí, vocês...  Prefere esperar ou já consegue fazer o teste?  Já começa a fazer o teste.  Então eu faço aí.

## meeting_id 21 - November 25, 2024 14:56

- [01:36] The chunk occurs during a discussion about organizing and managing tasks related to a project, indicating a need to regroup and reassess the next steps in the process.
  > :  И пересобираемся.  Теперь опять нужно сделать...

## meeting_id 22 - December 12, 2024 16:00

- [22:22] This chunk is part of a conversation between the AI Marketing Directory and Dmitry Grankin, discussing the structure and frequency of their potential partnership, specifically focusing on content planning and collaboration for VEXA, a meeting note-taking tool.
  > AI Marketing Directory:  Yeah, we'll detail out what the partnership would even look like, cause we would wanna meet with you like every two weeks, get an idea of what's going on with VEXA and he, product updates, any thing that you want to push to these audiences that we're building up for you. Any like industry news that you want to touch on like those kind of things. And then, like come up with a content plan based on our conversations for like the next two weeks and then we just kind of do it all over again yeah I feel like as of May
- [00:12] This chunk is part of a conversation between representatives of AI Marketing Directory and Dmitry Grankin, discussing the evolution of their respective companies and the changes they have made since their last interaction.
  > AI Marketing Directory:  So yeah, Phil and I started the company together.  We've made significant changes since we last spoke, which it sounds like you have as well, you know, which is what brought you to reach out to me in the first place.

## meeting_id 41 - December 06, 2024 10:31

- [00:00] This chunk is part of a larger discussion on partnerships and personal experiences related to business collaborations. It features multiple speakers sharing their stories about successful and unsuccessful partnerships, insights on managing relationships in business, and the challenges faced in various projects. The focus is on learning from past experiences and seeking reliable partners for future endeavors.
  > Dmitriy Grankin:  partnership but I have a lot to share and about how to finish some partnerships that are not I have a story about a bar where I started as an investor, we had another investor at 1 million, someone who committed to work and built Eversyn, but without money and just tour.  And we finished with me as someone who built Eversyn and someone who invested.  and so that we can have hope for the future, yeah.  So, my name is Stanislav.  about Alex, because he and, uh, one of the guys who was his wife, other partnerships, and as a matter of fact, He's one of his partners in eBasical, and Alex is also a member of the company.  It was a surprise to me, I didn't know that.  I'm actually a big fan of you and the performance.  How quickly can you launch a driver if you see something like that happen?  For people that I know, they've seen it happen.  as it will be.  Usually I have my own, my thinking, it's 99% the same story.  Now it's gonna be different, but I'm not gonna...  So right now, we used to have a marketing community But I don't work there anymore, I sold it.  guy who actually took on my responsibility. And that was the most successful partnership I got in my life because this company for 15 years. We never had a fight about money. At some point we had big about the direction of the company, where the company should go, but somehow we managed to resolve it and it was a huge point for the personal growth for me because I understood that it's not about me it's about us and it's it's a huge transformation in the head and I'm glad I had this experience I was way more miserable without this experience.  But it was not simple, for a few years we had this pressure.  I was coming to work and I was like, fuck, I need to see this guy again.  he wants this way i'm gonna go this way and i'm gonna force him to go my way and he was like no i'm gonna force him to go my way because i have no power And then at some point we decided to stop the war, so his territory will belong to him, Alliances and guests work together and now I think have great relations.  So, I think it was a great experience for that and the third guy bought some of my shirts and joined the company.  He is just amazing.  We are having great match, so what was fun is 11 in their case, so the company is moving even faster than the guys.  So those two partners were amazing, they're great guys, I have a lot of respect for them, and we grew a lot as human beings, so actually it's sustained, but I would it's not simple and not everyone can do it it's now i've been searching for a reliable partner for my next projects and partnerships that happened in the past six years were not i mean they were okay but not successful maybe because i've had certain expectations already quick and just give partners i was like okay why you guys are so why are you so irresponsible Yeah, it's like, oh, I don't want to go out there.  So this is something that I would score this out.  What to do next?  I want to thank you for what I wrote about the Shamanism in Peru, I spent 8 months in Sorry, what is the shaman?  Magician magician. Yeah, very much. We can like it it's kind of Of course, yeah.  But some of them are actually doing something very little.  Very, very little.  Yeah, I think I should shut up.  Now I have a marketing autopilot program, which is kind of a performance marketing thing and I'm also doing business in New Jersey.  What is your personal coaching for the past 15 years?  Business and personal life, my main product is time management and managing work for people.  I have so many projects and that's new commissioning.  Yeah.  Yeah.  That's pretty a lot.  This is the Spanish version.  Yeah.  You wrote a book about some of these exams that you started with on a quick basis.  He had really great partnerships with the ups and downs, but he managed to really break them.  He has a project of coaching, and he is a great marketer, and he's experienced the effects of the intensive years of his experience.  Let's come together, a little bit closer.  More or less, I'm right here.  My name is Andy.  I'm currently having two projects, the first one is a start-up in health tech, an IT start-up.  We help to design healthy habits not to track but to design it's like LEGO Constructor for new habits Another project is currency exchange I've changed crypto and I also help companies and individuals to optimize their corporate income tax.  So it's a different thing, it's fintech more or less.  Text, text.  Yeah, text, spin, whatever, a combination of those.  About partnerships.  almost 90% of my projects have been in partnership.  It was 2, it was 3, sometimes it was 4 even.  partners in the company uh i had also bad partnerships especially at the beginning now i i try to you know recruit and be slow for the for the partner partnership to see and You have maybe a small project at the beginning, you see how it works, and then if you go well, we can maybe extend your project. Now I am in partnership in my startup. We have five But those are attractive and after maybe an hour or two we really receive new perspective and third opinion inside ourselves.  it's just like it's really creative uh personally we don't find this like more how to engage with partnership quicker because now I have 3-4-5 months of becoming partners It takes time, maybe it's too long, but maybe there are some tools in Instagram to make partnerships, you know, to check if the partnership is visible much quicker.  That's fine, maybe everybody can go home and do what they want to do.  That's pretty much it.  I wanted to add something.  I've been working with Adrian for six months, and I highly recommend that you sit If I got it right, Andre, you have two big projects.  HealthTech is the newest one, I guess, that you have, that you're trying to kind of get people to pre-empt any issues so they can get into the motion of doing healthy things.  and uh and the second one you just help russians get their money out of russia not only no no but it turns out that other people want to do it too yeah i mean it's everyone is all over the world, so anyone can know how to say that in a different way.  It's not only for Russians, yeah, for everyone.  take your money out of the tax system which actually i want to say bitcoin but In independent money, as much as possible, taking third parties out of the equation.  All these intermediaries are there, so I'm very much for that.  So congratulations on this project and congratulations on the partnership.  I'm Christian, I'm Romanian, I lived in a few countries so far including the US, including Ireland and now I'm in Portugal for five years now and I started working on this project to help I help people with their personal finances through automation.  So automate people's bills to make sure that you don't You don't get overcharged or you don't get, you don't overpay for a contract or you don't miss a contract deadline or you don't overpay for your credits or loans or so on.  This is a very recent project that I started, I'm working on this for the last year, year and a half, and I launched it publicly about three weeks ago.  Three and a half, four weeks ago, and I'm doing it basically me and with a bit of help from my wife, with a bit of help of some Software developer friends that that I have we put together this kind of solution and this Because I'm looking for for a co-founder this is why I'm here looking for a co-founder looking to take this to the next level about partnerships as I can say I've been Build one of the largest partnership programs on the planet and I'm being modest about it.  Over a hundred thousand enterprise partnerships.  licensing models, royalties, technology integration.  So, started in 2009.  We developed some technologies, then we licensed those technologies and then we said why don't we create a partnership program so we can not only license these technologies but also help people integrate these technologies.  which is in their products, consumer products, and then sell these products as well.  So help these companies sell these products.  That's my partnership perspective, I guess, and experience.  We put this together out of need because we wanted our technology to go there outside in consumer goods.  So yeah, people built wireless speakers because of our technology and there were no wireless speakers It was before this technology came about, they built radio controlled smart home devices, thermostat lights and so on.  are the reason for my team is mostly the reason why in cars nowadays you have touch screen displays and a lot of people hate it, a lot of people like it because We wanted to put the ability to have your phone and all your controls and all your apps onto the car dashboard. And yeah, We created some health tracking devices as well, some technologies there for smart watches and licensed that as well.  Help people build their devices with this technology. So and then help people sell these devices But we I was working for a large company So we had a bit of a bit of backing there so it's not like I can say that I built this from scratch you know there's a team that there's a lot of people involved And then having a backing of a large multinational company, it proposes to the next level.  So, yeah, that's that's all I have now.  I have I'm doing my own thing, so.  Hopefully we can revolutionize and democratize the way that we deal with our bills, because Annoying me and I want to automate it so let's see if we can find partners that can bring and then find the way to work together that's all Christian. Okay, about Christian. Christian is Romanian who has lived in the U.S., Ireland and Portugal, is working on a project.  I don't remember this.  No, cheating, cheating.  What is this?  No, no, no, no, no.  This is his way.  Technology.  He's working on a project to help people manage personal finance through automation.  Automates bill payments to prevent overcharging and missing deadlines.  Started this project about one and a half years ago.  ...and launched it publicly three weeks ago, seeking for a co-founder to help to take the project to the next level.  Sorry about this guys, I've been testing this thing and I'll switch it off because my battery is going down.  But about me, I am Dmitry Grankin.  I've lived in Portugal for four years now.  And I founded VXA, VXAI, which is Google Meet Assistant.  ...which is building knowledge as we speak over Google Meet with customers, with team members.  And this thing works real-time during conversations and uh delivers a nice aftercare receipt dashboard which you can chat with any specific uh meeting or over multiple meetings to find out what's actually going on in the company It's got great potential for teams because bringing knowledge from multiple team members into one knowledge lake has significant potential.  This project started in Antler, which is an incubator.  a few cohorts here in Lisbon and we started with a co-founder but we broke up about a month after the project was finished.  And it took off right after the partnership broke up because I started up because I am a technical, I am a I was CTO of that company.  I've built probably almost the entire thing myself.  I would raise some capital.  I spent some bucks for development, but eventually now I cut off all the stuff and again it took off after because the technology shifted very fast.  dramatically recently uh i call it a lot of this of its cursor and uh it it delivers 10 10 times faster experience because I don't waste time on discussing things, on dailies, on retros, all the stuff I do, all these things myself.  that's about me thank you nice meeting you let me introduce you Dmitry he's living here for four years yes and he's also running ai assistant for google meet it is very good for teams that help you to understand What's happening in the company and he broke up his partner in this project and since we're going and faster because he don't need it didn't didn't need to um to have demos to She's spending time on synchronization, she's doing it herself.  So, about me, I have a presentation for you guys.  It is simple, but I like visual things.  There are many pictures and you tell me to read it, some things about me.  I'm 41 years old. You see here that I have a family with little son. He's Portuguese because he was born here and I have a dog. He was born in She is not here, so she is not a part of this talk.  I have 10 years experience in video production, this is my core expertise.  for several companies in this area for you later i will show and um and what i have to tell you about me also i'm uh I'm a skipper, yacht skipper, and my dream is to cross the Atlantic Ocean.  That's one of the reasons I'm here, because there is a good starting point to And now I'm going to tell you something about my business more and I will tell you about my tries of partnerships.  I'm not going to tell you that I'm a super expert in partnership.  I heard that all of you have some tries.  But I have many of them, I have some experience and I would like to share it with you and to discuss this is my This is my goal.  And this is a typical disclaimer, all I will tell you is not a recommendation, this is just my experience, please use it on your own risk.  I would appreciate if you share the same situations or maybe your Steps that you did in the same situation. It was very useful to share this experience so about my companies now i have two projects uh cult production and blastin studio uh and it is a Video production company and It's a computer graphics studio, all of them are, they produce high-end, they present commercial ads, high-end commercial ads.  and commercial content with the biggest mostly videos most videos advertising videos for big brands like You are my customer, did you know that? Actually first customer. Great, I appreciate it.  is one of my partners. I will tell about this partnership story. So we are working mostly in Russia with different Different big brands, national champions like Sberbank, I don't know, La Moda, Azon.  Also, we used to work with different international brands in the past you know so this is the question about about your partnership experience but you have told about it and so we Let's go to my story, I have 12 years experience in business, I have many attempts to build startups To try to do something and the first one was a partnership with my brother. We started many years ago.  Go that was my first business and what we do we just buy a floor In another in one city and sell it to our store is what?  What is the windmill produce for floor floor flower flower flower?  Yeah That is funny because my name is Melnikov and It is Melnika Pida, it means windmill man.  Millers, okay.  It's only in Austria you start there.  Yes, maybe, maybe, yes.  And we just started to do something with my brother it was fun and we learned a lot of things But things became, but they, sorry, but We had bad things to come. On our storage bags were appeared and they hate our goods and other problems.  And finally, my brother tied and told, so I'm gonna go to France.  A big holiday for two months and I can't tell him anything because he was my brother and I...  Yes, no.  No, boss, he is my brother.  He's dead to me now.  No, no, no, we have a good relationship.  Since that moment I was alone.  And I will start too.  And finally, he came back and I sell this business to him, my chair of the business.  And that was my first.  Exit, I mean, and it was a hundred years.  Yeah, that was my first round.  This is a very small business, but this is experience.  to do this with relatives and I don't recommend this way you have a success story guys but  section is not for me. The second one was a partnership as a side business. I explain what is it.  I had a friend, this is business with friends, I had a friend, we were, we are friends, and we were, and we tried...  To do something more and to run business from marketplaces together

## meeting_id 34 - November 25, 2024 12:54

- [08:09] The chunk occurs towards the end of a conversation where Dmitriy Grankin is discussing the functionality of a tool and its performance, indicating a moment of engagement or prompting further discussion.
  > Dmitriy Grankin:  А ну кащу, а то я вас умолчу.

## meeting_id 19 - December 12, 2024 09:59

- [03:30] The chunk is part of a conversation between Елизавета Иванова and Dmitry Grankin discussing the details of a meeting that took place on December 9th. They are clarifying the topics covered in that meeting, specifically distinguishing it from another meeting related to financial issues, and addressing discrepancies in how the meetings are recorded in their dashboard system.
  > Елизавета Иванова:  Да, 9-го это было, но это было не про финансовый труд.  а это была именно вот наша встреча. Другая встреча была.  Да, он ее почему-то сюда  не вывел, и я пошла на старый борт и на него пошла.  То бишь, вот он где-то здесь.  Вот, и он не дал норм оценку.  Так это у тебя же одна встреча 9-го числа, и там тоже одна встреча 9-го числа.  А здесь почему-то финансовые вопросы, ой, сейчас, трудности и планы.
- [04:20] The chunk is part of a conversation between Елизавета Иванова and Dmitry Grankin discussing a recent meeting on December 9th, where they reflect on the topics covered, including financial difficulties and personnel changes. They also address issues with the meeting's transcription and how recurring meetings may affect the documentation of discussions.
  > Елизавета Иванова:  это она и есть по идее не это старое что-то маленькие заказы отказы типа да у нас финансовые трудности типа что-то мы обсуждали а вот на это новой, 9-го, мы говорили про там что-то уже про Новый год, про то, что надо Диму переводить в другой отдел, да, и вот такие всякие штуки.  странная история может быть знаешь что у нас повторяющиеся встреча и он как бы не привязал к 9 декабря эту встречу, потому что она каждую неделю.  И он мог взять первичную дату.  Да нет.  А можешь нажать на транскрипт?  Правильно там, ну, вот там такая кнопочка есть, транскрипта.  Какая?  Вот чуть повыше, вот это, да.
- [00:00] The chunk is part of a conversation between Елизавета Иванова and Dmitry Grankin, where Елизавета discusses her recent health issues and the current status of a project, indicating a transition to discussing work-related topics.
  > Елизавета Иванова:  ...болела, вот, но сейчас уже нормально все.
- [03:02] The chunk occurs during a conversation between Dmitry Grankin and Eлизавета Иванова, where they discuss their experiences with a new dashboard and the challenges of finding past meeting records, specifically referencing a meeting with a person named Егор on December 9.
  > Dmitry Grankin:  Ну вот, я чуть зашла и думаю, а где моя встреча с Егором?  Нет, по-моему, ее тут нет.  нету и пошла в старый а как же такие там нету человек почему-то не знаю вот у меня была наша покажу А, в понедельник вроде. Вот эта встреча, вот я на ней что-то писала.  9 декабря.

## meeting_id 18 - November 19, 2024 11:45

- [01:28] The chunk occurs during a troubleshooting session where Dmitriy Grankin is expressing frustration about not being able to see reports and questioning the status of a meeting, indicating ongoing issues with a system or process he is working on.
  > Dmitriy Grankin:  Я не вижу отчетов.  Так и что? Диэктив митинг по-прежнему нету?
- [07:34] The chunk occurs during a troubleshooting session where Dmitriy Grankin is discussing issues related to system processes and functionality, specifically questioning why the "Worker" is not operational while expressing frustration about the overall performance and access to reports.
  > Dmitriy Grankin:  Так, а почему не Worker у нас не работает?  Субтитры сделал DimaTorzok Yeah.

## meeting_id 26 - December 03, 2024 11:05

- [19:07] This chunk is part of a conversation between Daniel and Dmitriy Grankin discussing the impact of AI on software development, the use of AI tools like Cursor, and the potential for sales of Dmitriy's product. They also touch on privacy concerns regarding data handling and the statistics of user interactions with the product.
  > Daniel:  Yeah, it seems like maybe AI is is completely changing the game in this as you as you as you know yeah the thing is that  people don't realize this the developers software engineers they don't realize because they They perceive it as a linear development.  But I, myself, being with this thing, I've been training machine learning algorithms since 2012.  I understand it's a logarithmic growth and I see the leaps, it's a growth.  So I've been, you know, playing around with all this stuff heavily And I do, because I know that the next step will probably change everything.  And these steps actually happened like weeks ago, like weeks ago.  Not many people understand.  understand this even they heard about it they they tell not to use it because they have some ideas about it are you using cursor or are you using just uh I'm using Cursor. It makes life much easier.  What agent are you using? Cloud?  Yes, mostly.  Mostly 3.5 Sonnet. It's kind of optimized for coding.  Yeah, it's really powerful.  for coding cloud is the best yeah so um it's pretty weird but it's a matter of fact as a  matter of fact yeah maybe maybe you should spend some  some time finding a good uh uh let's say person from uh marketing or things like that that will help you to get have more sales as long as i understood you are You are still not selling the product, right?  Yeah, yes, but it looks like it will be a lot of quick sales.  Because there is a lot of people who are saying the same thing.  They are using TLDB, they are using Tactic, they say about this language problem.  Yeah, yeah.  Yeah, you have something very special.  Yeah, I think, yeah, let me use it a little bit and then we can talk more in a few minutes.  in a few days or in a couple of weeks sure sure okay yeah  as soon as you talk to my product i'm happy okay  do so are you able to see uh so my conversations they are keep they are kept in my in my machine or they are sent to any server or something like that your conversation  are in the database I don't see your conversations no one is  seeing your conversation  I see statistics about about um like uh that i i i saw you that you had the conversation  okay you have some metadata about my conversation yeah i i this this i  i research statistics about how clients interact with my product like today like we got a uh Today we have 26 conversations, 17 active weekly users.  Yeah, one thing that I believe is a powerful...
- [02:13] This chunk is part of a conversation between Dmitriy Grankin and Daniel discussing a language transcription plugin that translates conversations in real-time. They explore its competitive advantages, including its ability to handle multiple languages, and address minor bugs encountered during its use.
  > Dmitriy Grankin:  yes so it's and it works in any language right yes Sim, sim, se você falar português agora, vai mudar para português, que é um competitive advantage over some of the competitors we have like tactic uh these guys they yeah that guys they can do this look it's a it's there's a small little bug over there  little bug yeah because it's let's say it changed the transcription to portuguese and it is translating to portuguese yes yes it happens it's a little bit over  like uh too smart sometimes yeah this happens overfitting yeah or um yeah overdoing its job yeah okay but it's really interesting yeah and uh what  is the name of the plugin how can I install it here to test

## meeting_id 2 - November 24, 2024 20:25

- [00:16] The document appears to involve a conversation about a pending decision or response, with the chunk indicating a moment of inquiry or anticipation regarding the situation being discussed.
  > :  Сейчас мы посмотрим.
- [00:27] The document appears to involve a conversation about a request or query that is not being fulfilled, with a specific focus on the reasons for the lack of response or resolution.
  > :  Решение пока не отдает. Почему не отдает?

## meeting_id 11 - November 25, 2024 21:24

- [01:08] The chunk is part of a conversation by Dmitriy Grankin discussing the update process of a system or application, expressing curiosity about its functionality and current status, amidst a broader dialogue about active meetings and their handling.
  > Dmitriy Grankin:  Смотрим сейчас как эта штука будет обновляться Если будет как-нибудь обновляться, мы посмотрим.  она будет обновляться очень интересно Как она будет обновляться или не будет обновляться.  Пока никак не обновляется.  Наверное, это связано с тем, что...  Ну, на Бальтини обновляется.  Muito bom.
- [02:52] The chunk appears to be a continuation of a discussion about the handling of active meetings, with the speaker expressing frustration and a desire to focus on their own tasks rather than dealing with the issues related to these meetings.
  > :  что мы будем делать с этим  Отход до 254 встречного сапога.  нельзя  занимаюсь своими делами

## meeting_id 32 - December 02, 2024 15:30

- [00:00] The chunk is part of a conversation between Dmitriy Grankin and Igor Lyubimov, where they discuss their recent activities, specifically focusing on coding and work-related tasks.
  > :  Да, Игорь, привет.  Что, как дела?  Ничего, вот сидел несколько недель, кодил.

## meeting_id 10 - November 19, 2024 18:20

- [00:00] The document appears to capture a brief dialogue or exchange, where one speaker is inquiring about a situation or event, indicating a moment of confusion or curiosity.
  > :  Что происходит?
- [00:02] The document appears to capture a moment of uncertainty or hesitation in a conversation, with the chunk reflecting a pause or lack of clarity from the speaker.
  > :  Эээ...

## meeting_id 20 - November 19, 2024 12:03

- [01:54] The chunk captures Dmitriy Grankin's reaction to a disruption during an active meeting, expressing confusion and concern about the situation.
  > Dmitriy Grankin:  А-а-а-а-а-ам!  Что происходит? Почему?
- [06:21] The chunk is part of a conversation where Dmitriy Grankin expresses confusion and frustration about a meeting he cannot see, followed by a discussion about updating and restarting a system or application.
  > :  Хотел перезапустить.  Чуть-чуть.
- [00:00] The chunk is part of a conversation where Dmitriy Grankin expresses confusion and frustration about a meeting he cannot see, while also mentioning a topic related to "Интекс монета," indicating a discussion about a specific subject or project.
  > Dmitriy Grankin:  Интекс монета.

## meeting_id 37 - December 16, 2024 17:21

- [00:00] This chunk is part of a conversation between Carine do Amaral de La Puente Machado and Adilene Silva, discussing the conclusion of a cycle and the transition to a new phase, likely related to a work or educational program. It sets the tone for a meeting focused on operational updates, scheduling, and participant engagement as the year ends.
  > Carine do Amaral de La Puente Machado:  com a sensação de que estamos conectados, encerrando esse ciclo e começando o próximo.  próximo embrego, né?  Ótimo. Vou compartilhar minha tela aqui rapidinho, e a gente tenta ser bem objetivo aqui, que eu sei que as agendas estão desesperadoras.  Meu Deus, final de ano.  Tá bom.  e fora correria você tá bem aí eu tô bem tô ótima gente muito  obrigado por perguntar mas tô bem assim fora o ao lado do ano porque né também. Tá bom, que bom.

## meeting_id 15 - November 19, 2024 10:08

- [02:22] The document appears to be a discussion about testing and updates related to a system or application, with specific references to the presence or absence of features and the need for the latest updates. The chunk emphasizes the necessity of having the most recent update, indicating a concern about missing information or functionality.
  > :  Нам нужно иметь последнее обновление. Здесь нет последнего обновления.
- [00:00] The document appears to be a conversation or discussion involving Dmitriy Grankin, where he is testing a system or application and noting the presence or absence of certain features or updates. The chunk specifically captures his remarks about testing and the results observed.
  > Dmitriy Grankin:  еще раз возьмусь потестировать сразу появилась 1008 здесь сразу появился Здесь не появилось, а здесь не появилось мотомопитест.  Mm-hmm.
- [01:26] The document appears to be a conversation or notes regarding testing and updates related to a system or application, with specific references to features, speakers, and the need for the latest updates. The chunk reflects a statement about the absence of certain openings or features, indicating a focus on the current state of the system being discussed.
  > :  никакие открытия пусть их нету есть поле

## meeting_id 7 - November 24, 2024 21:54

- [00:00] The chunk expresses a desire for repetition and emphasizes the need to revisit a previous point or action, indicating a focus on clarity or understanding in the ongoing discussion.
  > :  давай еще раз  еще раз еще раз еще раз

## meeting_id 12 - November 25, 2024 19:45

- [28:31] The chunk is part of a conversation involving Dmitriy Grankin and others discussing technical aspects of a project, including updates and execution issues related to a system or software, with references to monitoring and status checks.
  > :  идея а совсем другое дело исполнение  совсем другое дело  Ну че теперь, TX0?  чем те, по которым не было обновлений в течение...
- [27:09] The chunk occurs during a conversation where Dmitriy Grankin and others are discussing the status of a project or system, with some light banter and confusion about greetings and updates, reflecting a casual and somewhat chaotic atmosphere.
  > :  Ну это кто сказал добрый день? Я сказал добрый день? Такого не говори.  минус кем-то как всегда террорист
- [28:11] The chunk is part of a conversation where Dmitriy Grankin and others discuss technical aspects of a project, including monitoring statuses and updates, with references to specific versions and functionalities.
  > :  Еще интересно, что у меня тут 9.13.  должен был здесь появится по идее в одно дело и
- [26:25] The chunk is part of a conversation where Dmitriy Grankin and others are discussing the status of a technical process, specifically monitoring updates and system responses, indicating a testing or troubleshooting scenario.
  > :  Ещё раз попробуем, запускаем. Вот у нас пошёл...  Пошел звоночек и ждем обновлений.  Продолжение следует...
- [00:40] The chunk is part of a conversation by Dmitriy Grankin, discussing the status of a project or system, expressing surprise that something is not failing or falling apart, and reflecting on the situation in a casual manner.
  > Dmitriy Grankin:  И не валится.  Ну вот это что, всё-таки был такой...
- [03:37] The chunk is part of a conversation by Dmitriy Grankin discussing technical aspects of a project, specifically mentioning an endpoint and a component referred to as "червячонка" (little worm), amidst a series of tests and updates related to a monitoring tool.
  > Dmitriy Grankin:  мульти и что появилась у нас так вот end-point а Yeah!  Теперь мы хотим...  взять этого червячонка.
- [01:52] The chunk occurs during a conversation where Dmitriy Grankin is testing and discussing various technical aspects, including monitoring and updates, before casually saying "Tchau, tchau," indicating a transition or conclusion in the dialogue.
  > Dmitriy Grankin:  Tchau, tchau.
- [00:00] The chunk is part of a conversation by Dmitriy Grankin, where he is testing a system or application, expressing frustration about its stability, and engaging in a casual dialogue with others about the status and functionality of the system.
  > Dmitriy Grankin:  Тест, тест, тест, тест, тест.  С. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС Так, что ж отвалится все-таки, да?
- [02:00] The chunk is part of a conversation by Dmitriy Grankin, discussing the status of a project or system, expressing curiosity about its functionality, and indicating a desire to investigate further.
  > Dmitriy Grankin:  Не падает. Пойдем посмотрим, что делает вот этот вот.  Как у нас выглядит с тверочком.  Ну ладно...
- [29:40] The chunk is part of a conversation involving Dmitriy Grankin and others discussing technical aspects of a project, including testing and monitoring statuses, with references to updates and interactions during a meeting.
  > :  Это все та же встреча.

## meeting_id 14 - December 05, 2024 19:00

- [00:00] The chunk occurs at the beginning of a conversation between Lara Vargas and Dmitry Grankin, where they are confirming their audio connection before discussing various topics, including their locations, weather, and plans related to VEXA, a product they are working on.
  > :  Good can you hear me? Well, okay. Yes. Yes. Yes. Yeah. Perfect.
- [00:34] The chunk occurs during a conversation between Lara Vargas and Dmitry Grankin, where they discuss their locations and experiences with the cold weather in Portugal, highlighting Lara's Brazilian background and her unfamiliarity with the colder climate.
  > Lara Vargas:  for a brazilian like me where uh what part of brazil are you from i'm from rio so yeah i'm not used to this I think the greys is already a lot for me, I mean.
- [24:49] This chunk is part of a conversation between Lara Vargas and Dmitry Grankin, where Lara discusses her recent work on a Black Friday marketing campaign at Buildtrix, her preference for B2C over B2B marketing, and her desire to return to a more customer-focused role. She reflects on the differences in content creation between her previous job and her experience with Vexa, emphasizing the fulfillment she finds in understanding and engaging with customers directly.
  > Lara Vargas:  So for the last, for example, the month of October and November, I couldn't focus much on content posting because we were focusing on creating our black friday campaign so i was doing a marketing funnel to create this black friday campaign so i spent the last last two months doing that yeah i will send you the page for build tricks and why are you living in the company Because I want to work more. I don't really like working B2B, I have to say. I prefer working B2C. It makes me happier. It makes me...  I prefer talking to the customer, I think that I have a knack for understanding the customer, you know, I feel that I know what people want.  and i like talking to them about it i like feel i like the pleasure it gets me when i understand them uh means like you're more like  on a product manager management site like talking to customers and customer development interviews all this all this stuff  Sim, eu quero dizer, quando eu estava, por exemplo, quando no Beatrix o conteúdo é muito mais impessoal, sabe, é muito mais...  Mesmo o copywriting, o que eu tenho que dizer ao cliente é muito mais impessoal.  O valor, o benefício que eu tenho que oferecer.  is more impersonal. It's not how that person will improve its life, it's how the company will improve their life. And for example, when I was working with you guys in Vexa, that was something way more fulfilling for me and that's what what showed me that i actually wanted to go more uh go back into b2c so i'm yeah i'm actually looking for stuff in more related to b2c and you perceive the access to b2c product i think you guys have both right i think that there is uh the free use part there is the the paid part for the consumer but there is also the big the big part net right which is where you get most i think of the it's where the the the mark the big value of the company will be right is selling to big companies but yeah i think that i believe that as as a startup fax will first grow in the consumer right and
- [39:51] This chunk occurs towards the end of a conversation between Dmitry Grankin and Lara Vargas, where they discuss scheduling a follow-up meeting, the use of Google Meet for calls, and issues related to user access and premium subscriptions for the VEXA platform.
  > Dmitry Grankin:  Okay, Laura, I think we can have another meeting next week.  Okay.  Just...  Just think about it.  Yeah, I will.  I will start researching what kind of different uses we can propose.  and get back to you and I will I will check on your on your calendar I will already schedule maybe for Tuesday or Wednesday if that's okay for you yeah  And do you tend to have these Google Meet calls?  I'm sorry?  Do you have a lot of Google Meet calls?  I only take Google Meet.  I only use Google Meet.  Is it, do you have many of those?  Yeah, sometimes.  Yeah, some days I have, some days I have full day, some days I don't.  don't have any quotes here so why don't uh why don't you select something because now i'm on my ipad i was gonna ask you that we need an integration i but i always use vex dmitry i'm always with vex it's just that i i don't have my laptop right now i'm only with my bed oh i don't see you because maybe i've  you to the list of insiders that's like really i don't see you as a as a user because i just really uh yeah removed some people who Who I consider like not being a user.  Really?  Larger embargoes, I'm not being user.  I use it so much, I even I got really upset the other day.  I was going to tell you this because I recorded the meeting.  I think it was even with you guys, the meeting that you were showing me that was in rushing and I couldn't find it.  my dashboard you couldn't find anything that we had no I couldn't find it I got  rid of But I it was recorded because I saw it getting recorded at the time  Let me find it. I'll share it with you if I have it. I Saw this meeting I I show this meeting to show you this meeting.  Yeah, I, I, I open my large Vargas.  I have my let me see if I can.  share with you i found i found you yeah yes i found that i've added the action i've actually added your uh yes your account as and i  yeah as a test in account account okay another thing i meant to ask you sorry dimitri i'll let you go but this is the last thing uh lucas asked asked me how to see if he is a paid user because we negotiated with him premium for a year and he's he was asking me I think that  all the users are premium users but I can assure him that as soon as we have  have payment uh his one year will start from that point in time okay okay okay if it works for you  okay i will let him know i will let him know  okay great okay I'll talk to you soon okay what's so great to talk to you one  okay bye-bye you too bye-bye
- [00:05] The chunk occurs at the beginning of a conversation between Lara Vargas and Dmitry Grankin, where they discuss their current locations, specifically mentioning Lisbon and the cold weather, setting a casual tone before transitioning into more detailed discussions about business and personal interests.
  > Lara Vargas:  I'm on my iPad, so I just want to check on Lisbon.  Yeah, I am. Are you as well?  Yes, on Belas Clube do Campo, not far from Lisbon.  Oh, okay, nice. It's been very cold, isn't it?
- [08:19] This chunk is part of a conversation between Dmitry Grankin and Lara Vargas discussing the performance and user retention of their product, VEXA, particularly highlighting its appeal to Brazilian users due to its multilingual capabilities.
  > Dmitry Grankin:  Yeah, and it becomes, I don't know, it becomes a market actually we've got we've got competitors uh we've our products got retention with users and steady influx of new users daily and actually quite a lot of from Brazil I don't have a good analytic system to understand what is actually the source but these links don't happen UTMs. But as a matter of fact, there's a lot of Brazilians, maybe because of the multilingual capabilities of VEXA.  what people like about it is that it seriously switches from one language to
- [08:41] This chunk occurs during a conversation between Lara Vargas and Dmitry Grankin, where they discuss the impact of influencer marketing on subscriptions for their product, specifically referencing an influencer named Renata and her promotion to her community. They mention specific dates related to the influencer's activities and the subsequent relaunch of their product.
  > Lara Vargas:  another during call that's great that's great and did you get many any subscriptions after the um after the sign up with from renato  sign up with renato what was that about what  Renata was the second influencer that we chose and she had a community of about close to 600 people actually and she promoted  directly to them what was the point in time when this was it was I think it was on the 23rd of  September and then they read they relaunched on the 28th let me it was on the 20th of november and then on the 28th they announced again november yeah uh 20th of november  And?  And then the 28th, 28th.  Okay. So I don't know.

## meeting_id 3 - December 16, 2024 17:30

- [01:00] This chunk is a conversation between Gavin Jung and Tatiana Sukhova, where they discuss their weekends, personal challenges, and work-related updates, including project planning and team dynamics. It serves as an informal introduction to their ongoing collaboration and sets the stage for more detailed discussions about engineering and product development later in the document.
  > Gavin Jung:  hi good morning good morning it's almost dark  how was your weekend um this one this one was very uh very slow because my husband is sick Oh, he's still sick.  Yeah, he's pretty sick.  Oh, no.  And he lost his smelling abilities, and we think it's COVID again.  Oh, no.  But I am still...  i'm still fine which is surprising so it's good news i'm still fine yeah and i was able to send my son to school today finally which is also very helpful  oh good yeah so what about you it was kind of a busy weekend my wife's we went out for my wife's birthday on friday with a bunch of friends and so we went uh we went karaoke yeah and she had she had asked um for this type of get together like two years ago and so we were finally able to pull it together with all the all the people she wanted there so that was fun uh but that made saturday kind of slow and then And so, yeah, we kind of laid low on Saturday and then Sunday had some friends invited us over for dinner to get the families together because they.  they had missed Thanksgiving because they were away.  So they wanted to cook a Thanksgiving meal, but they also didn't want to make that much food unless other people could eat it.  So we kind of had that.  like a like a belated thanksgiving meal with them uh last night so yeah it was good overall overall i would say more relaxing than being busy but um but still uh a pretty memorable one awesome yeah yeah that's cool i play i play puddle every weekend  and i did it yesterday as well even though i felt like i didn't feel well something because i'm like like every day i'm getting ready to get sick because every single person including my daughter already she is sick again as well i'm like every day yeah but then i just decided to go and to try to push it out kind of to make myself exercising so that i run my blog yeah and kind of try to struggle against and i think it's weird because today i work work up and realize that i'm fine again yeah so that's good yeah that's good  yeah there's something to that for sure to keep stay active to kind of limit the ability to to get sick and things like that but yeah i mean as parents we're like constantly surrounded by sickness. I feel like with kids, especially this time of year, right? With kids, you know, being exposed to other kids at school and all that kind of stuff. So I think it's a good mindset.  in portugal it's very cold inside of the park like we have freezing we have oh no there is no central heating and when outside it's getting cold more like 12, 7, 8 Celsius.  Anyways, like colder than summer.  Yes, I understand.  It's freezing inside.  Freezing right now.  okay yeah how are you doing any news anything cool bad no i mean um  i've been uh there's a lot of i think we're trying to figure out um on the engineering and product side, like how do we get enough ahead to where we're not doing, like trying to figure out last bit, like not just lining up the work right before it needs to be implemented, but there's some buffer time between that.  And I told, and Eric understands already and AJ knows that you and I are trying to do all this work here so that we can build out a bunch like a slightly bigger buffer for work for the engineers to focus on from a feature development standpoint.  So, so we're trying to get the, you know, get as much of that done before the end of the year.  here so that there's some room.  And then eventually, we're planning things like weeks in advance rather than the week that we're doing that.  There's also conversations about whether or not a two-week cycle is always the best thing to implement.  Maybe we do shorter cycles and break up the work even smaller if we can.  So, yeah, but we have some thoughts around that too.  And then overall, I think, I think, you know, yeah, we're, we're, you know, gearing up to like, be very, um, intentional with like, um, how we go about our work.  And so AJ has some, he's been working on like, kind of like documentation on like the company values and the vision and going from there.
- [07:00] This chunk is part of a conversation between Gavin Jung and Tatiana Sukhova discussing project management and prioritization of tasks related to product development. They are reviewing user stories, feedback, and the organization of tasks in Linear and Notion, aiming to establish timelines and clarify the order of issues to address as they approach the end of the year.
  > Gavin Jung:  So overall, I think we're, you know, last week we ended a good note in that sense and um and yeah i just want to i'm trying to get a sense of like uh after you and i meet today where we are on the product and then i can start to figure out timelines for for the work that we've identified so far, and then going from there.  Okay, sounds good.  Yeah, trying to close out the year very strong.  Yes, let's try.  so generally speaking I can't say that I'm far but I started off your user source that you lined up in linear and I was like the first question that I had is actually there are some that live in notion right that are not in the in linear and not yet yeah last part in linear which I guess you wanted to prioritize over the rest of the ones that in the notion is that yes right okay yeah uh so today i i started like i ran through all of them i have some uh feedback and i started on the first one that i decided must be probably the number one because the order is not quite or is it the order you want want me to tackle these the issues um i don't know if the order in  lanier is logical but i wrote it out in this in notion in a logical way okay so that like you can reference it but but yeah I mean I maybe I could have been more clear about that but yeah linear kind of did it like despite what order I put it in like entered it into linear it's I think it just assigned based on like, I don't know if it's alphabetical or something like that.  So yeah, I don't know the way, what view you're looking at in the project, but I imagine it's, it's not nothing indicate that it's very clear in terms of what order it should be from a priority perspective let's look together so okay what oh what i'm um what I'm looking at no at linear this is the order okay not the order the least  and this is the notion where let's say at product model right the last one here  Yeah, it's yes, you're right 20

## meeting_id 16 - December 05, 2024 13:59

- [24:26] This chunk occurs during a conversation between Dmitry Grankin and Sunny Wyrzykowski, where Dmitry discusses the advantages of his product, Vexa, in comparison to another product, Fathom. He highlights Vexa's unique features, such as its silent operation during calls and multilingual capabilities, while also emphasizing its ability to build collective knowledge from meetings. This discussion is part of a broader dialogue about business strategies and user acquisition.
  > Dmitry Grankin:  Okay, so you are absolutely happy about this product nothing really bothers you okay cool that's one of a few products with which i i hear i hear that okay great i i encourage you to to try out vexa because um this what are the competitive advantage on which people are telling me not what I am telling people, but my users are telling, is that it stays silent on the call, doesn't participate.  As a meeting participant, there's some...  noise people i don't know why actually but as a matter of fact and as i mentioned earlier it's multilingual and it's seamlessly um switching from one language to another and as what I'm telling the users you know they are telling me is that we are building knowledge with that so we can research not just a specific meeting after a call, but we can actually ask general questions about who are our competitors, what users say about us.  what we've been doing the last two weeks and yeah creating reports that are derived from the full body of uh collective knowledge how how does it uh how does it sounds to you
- [26:22] This chunk occurs at the end of a conversation between Sunny Wyrzykowski, Nora Köppling, and Dmitry Grankin, where they discuss Dmitry's product and potential consulting services. It signifies the conclusion of the meeting, with expressions of gratitude and a positive farewell.
  > :  great okay pleasure to meet you dimitri thanks guys thank you
- [00:00] This chunk is the opening dialogue of a business meeting where Sunny introduces herself and Nora to Dmitry, inquiring about their locations before discussing the agenda of the call.
  > :  Hi, hi guys.  Notice.  how are you good how are you doing pretty good so my name is sunny joining here is nora i'm not sure if you guys already met um but where are you where are you calling in from today

## meeting_id 27 - December 03, 2024 13:01

- [13:07] This chunk occurs towards the end of a conversation between Anna Pavlova and Dmitry Grankin, where they discuss scheduling a meeting and sharing contact information after a series of networking events related to a business club in Lisbon.
  > Anna Pavlova (Lupenko):  там может быть в районе 12 сейчас может быть если не пугает вот либо можно в течение дня или ближе может быть к вечеру если вы собираетесь лиссабон по каким-то делам да можно просто попробовать попробовать пересечься вот ну либо я могу пригласить на пятницу как раз на наш завтрак в качестве плюс один у нас есть такое условие что что резидент может пригласить плюс один одного человека вот заодно после этой встречи можете попить кофе с нашими сил в пятницу а во сколько здесь С 10 до 12 это Амарейрош, если на машине, то очень удобно.  Конус Гринстон. Ну, я скину информацию, вот.  Угу.  а я добавлю так вроде да у меня ничего нету хорошо тогда  Тогда напишите мне, пожалуйста, свой Телеграм, я напишу, скину туда информацию по встрече, вот, и на почту тоже кину приглашение на мероприятие, вот.  Вот, а там же кину потом в телеграм-контакт Александра, чтобы вы друг друга нашли.  Я, может быть, тоже буду, постараюсь быть.  Ну, ладно.  Ну, класс, рад был познакомиться. Давайте так.  Да, взаимно. Да, спасибо. До скорой встречи. До свидания.  Куда кинуть, куда кинуть? Я, может быть, прямо здесь сюда кину, чтобы?  да можно или в линкедин где мы общались целом тоже можно хорошо такой  такое количество сообщений после веб-саммита  неразобранных, что я подозреваю,  что у вас еще хуже.  Это точно. Но  Но приходится отвечать в рамках своих обязанностей.  Да, потому что я после веб-сайта ушел кодить, и там вообще прям совсем...  прям совсем ну ладно а я тогда сейчас кинул телеграм-канал договорились я сразу отвечу с

## meeting_id 29 - December 16, 2024 19:28

- [00:04] The chunk "Acabou." is part of a dialogue where multiple speakers are engaged in a conversation, expressing confusion and seeking clarification about what has ended.
  > :  Acabou.


In [11]:
chat_manager = UnifiedChatManager(
        qdrant_engine=qdrant_engine,
        es_engine=es_engine
    )

In [12]:
chat_manager.messages

[]

In [13]:

@jupyter_stream_output(markdown=True)
async def unified_chat(
    query: str,
    meeting_id: Optional[UUID] = None,
    entities: Optional[List[str]] = None,
    thread_id: Optional[str] = None,
    temperature: float = 0.7
):
    """
    Unified chat function supporting both single-meeting and multi-meeting search
    """
    
    async for result in chat_manager.chat(
        user_id=user_id,
        query=query,
        meeting_id=meeting_id,
        entities=entities,
        thread_id=thread_id,
        temperature=temperature
    ):
        if 'chunk' in result:
            yield result['chunk']
        elif 'linked_output' in result:
            yield result

In [14]:
thread_id=None

In [18]:
r = await unified_chat(
    query="put it all together",
    thread_id=thread_id
)
thread_id = r['thread_id']

Certainly! Here’s a comprehensive overview of VEXA, integrating all the discussed aspects:

---

### VEXA Overview

**VEXA** is an innovative meeting note-taking tool designed to enhance productivity and knowledge management during online meetings. It leverages advanced features to provide users with a seamless experience, making it a valuable asset for individuals and organizations alike.

#### Key Features:

1. **Real-Time Transcription**: VEXA captures conversations in real-time, allowing users to focus on discussions without the distraction of note-taking. This feature ensures that important points are recorded accurately.

2. **Multilingual Capabilities**: The tool can automatically switch between languages during meetings, accommodating diverse teams and international participants. This flexibility enhances communication and inclusivity.

3. **Knowledge Building**: VEXA enables users to build a repository of knowledge from their meetings. Users can access transcripts, derive insights, and follow up on action points, making it easier to retain and utilize information.

4. **User-Friendly Dashboard**: The newly introduced dashboard organizes insights from meetings, allowing users to research specific topics or individuals mentioned in discussions. This feature enhances user engagement and facilitates knowledge sharing.

5. **Integration Potential**: VEXA is exploring integrations with platforms like Google Meet and Microsoft Teams to streamline user experience and expand its reach within organizations.

6. **User Feedback and Iteration**: Continuous improvement is a priority for VEXA. Gathering user feedback helps inform updates and feature enhancements, ensuring the tool meets user needs effectively.

7. **Content Creation and SEO**: VEXA aims to create content that addresses specific user needs, such as acquiring transcripts from YouTube. This strategy not only attracts organic traffic but also positions VEXA as a solution to common problems.

8. **Partnerships and Collaborations**: Exploring partnerships with complementary tools can enhance VEXA's capabilities. Collaborations can create comprehensive solutions that benefit users.

9. **Marketing Strategies**: Engaging micro-influencers and utilizing social media for user testimonials can amplify VEXA's reach. Targeted outreach can help build brand awareness and attract new users.

10. **Analytics and User Insights**: Implementing analytics tools to track user behavior provides insights into feature usage and user engagement, guiding future developments.

11. **Security and Privacy**: Addressing security and privacy concerns is essential, especially for enterprise users. Offering solutions like deploying VEXA on client infrastructure can alleviate data security concerns.

#### Strategic Focus:

- **Dual Market Approach**: VEXA targets both individual users and companies, allowing for a strong user base while addressing the needs of larger organizations.
- **Influencer Marketing**: Leveraging influencers to promote VEXA, particularly in markets like Brazil, can enhance brand visibility and user engagement.
- **Continuous Improvement**: By iterating on user feedback and exploring new features, VEXA can adapt to changing user needs and market demands.

---

In summary, VEXA is positioned as a powerful tool for enhancing meeting productivity and knowledge management. Its unique features, strategic marketing approaches, and focus on user needs can significantly contribute to its growth and user adoption. 

If you have any further questions or specific areas you'd like to explore, feel free to ask!

In [20]:
r['linked_output']

'In the recent meeting between you, Dmitriy Grankin, and Jasper Vanuytrecht, several key topics were discussed regarding your product related to meeting summaries and knowledge management. Here’s a summary of the main points:\n\n1. **Product Features and User Base**: You highlighted the functionality of your product, which includes a dashboard that provides summaries and a chat interface similar to ChatGPT. This allows users to generate insights based on specific meetings or a collection of meetings. You mentioned that the product is currently in a free trial phase, attracting a stable user base with minimal marketing efforts ([1](/meeting/2a40c363-c52b-4f89-8719-e0be87591485)).\n\n2. **Sales and Marketing Strategy**: Jasper emphasized the importance of identifying target customers and crafting effective outreach messages. He suggested that a strategic approach to sales and marketing is crucial for growth, particularly for tools addressing enterprise privacy and knowledge management ch

In [13]:
meeting_ids = ['q','w']

In [14]:
url_dict = {
    idx+1: f"/meeting/{mid}"
    for idx, mid in enumerate(meeting_ids) if mid
} if meeting_ids else {}

In [15]:
url_dict

{1: '/meeting/q', 2: '/meeting/w'}

In [16]:
import re

In [17]:
async def embed_links(text: str, url_dict: dict) -> str:
    # Find all numeric references in the text
    references = re.findall(r'\[(\d+)\]', text)
    
    # Replace each reference with the corresponding URL based on position
    for idx, ref in enumerate(references, 1):
        if idx in url_dict:
            text = text.replace(f'[{ref}]', f'[{ref}]({url_dict[idx]})')
    return text

In [18]:
await embed_links(r['linked_output'],url_dict)

'In the recent meeting between you, Dmitriy Grankin, and Jasper Vanuytrecht, several key topics were discussed regarding your product related to meeting summaries and knowledge management. Here’s a summary of the main points:\n\n1. **Product Features and User Base**: You highlighted the functionality of your product, which includes a dashboard that provides summaries and a chat interface similar to ChatGPT. This allows users to generate insights based on specific meetings or a collection of meetings. You mentioned that the product is currently in a free trial phase, attracting a stable user base with minimal marketing efforts ([1](/meeting/q)(/meeting/2a40c363-c52b-4f89-8719-e0be87591485)).\n\n2. **Sales and Marketing Strategy**: Jasper emphasized the importance of identifying target customers and crafting effective outreach messages. He suggested that a strategic approach to sales and marketing is crucial for growth, particularly for tools addressing enterprise privacy and knowledge m

In [19]:
chat_manager = SearchChatManager()

NameError: name 'SearchChatManager' is not defined

In [65]:
from helpers import jupyter_stream_output
from typing import Optional

In [ ]:
from typing import Optional
from helpers import jupyter_stream_output
from search import SearchAssistant

# Initialize managers
search_assistant = SearchAssistant()
chat_manager = SearchChatManager()

In [ ]:

@jupyter_stream_output(markdown=True)
async def stream_chat(user_id: str, query: str, thread_id: Optional[str] = None):
    async for result in chat_manager.chat(
        user_id=user_id,
        query=query,
        search_assistant=search_assistant,
        thread_id=thread_id
    ):
        if 'chunk' in result:
            yield result['chunk']
        elif 'output' in result:
            yield result['output']

# Usage
await stream_chat(user_id=user_id, query="андрей")

In [ ]:

# Call the decorated function
await stream_chat(user_id=user_id, query="маркетинг")

In [ ]:
await stream_chat(user_id=user_id,query="маркетинг")

In [ ]:
SearchChatManager

In [21]:
r = await ParsedSearchRequest.parse_request("маркетинг")
queries = r.search_queries

In [ ]:
queries

In [40]:
search_results = [
            await search_assistant.search(q.query, user_id=user_id) 
            for q in queries
        ]

In [41]:
import pandas as pd

In [42]:
search_results = pd.concat(search_results)
search_results = search_results.drop(columns=['vector_scores', 'exact_matches'])\
    .drop_duplicates(subset=['topic_name', 'speaker_name', 'summary', 'details', 'meeting_id'])

# Prepare context using search assistant's methods
prepared_df, indexed_meetings = search_assistant.prep_context(search_results)

In [43]:
columns_to_drop = ['timestamp', 'vector_scores', 'exact_matches', 'source', 'score', 'meeting_id']
existing_columns = [col for col in columns_to_drop if col in prepared_df.columns]
context_prepared = prepared_df.drop(columns=existing_columns)

context = context_prepared.to_markdown(index=False) if not prepared_df.empty else "No relevant context found."
        

In [45]:
context = await context_provider._get_raw_context(user_id=user_id,query="маркетинг")

In [ ]:
context

In [ ]:
context_provider.url_dict

In [ ]:
await search_assistant.search(query="vexa",user_id=user_id)

In [ ]:
context = await context_provider.get_context(user_id=user_id,query="vexa")

In [ ]:
context

In [ ]:

meetings = await vexa.get_meetings()

In [6]:
from chat import MeetingContextProvider,MeetingChatManager

In [7]:
from psql_helpers import get_session,get_meeting_by_id

In [8]:
from helpers import jupyter_stream_output

In [21]:
meeting_id = 'dbf8ad27-649c-4a16-b5a1-5ab3db002bd9'

In [ ]:
from uuid import UUID
meeting_id = UUID(meeting_id)

In [ ]:
meeting_id

In [23]:
query = 'summary'

In [24]:
thread_id = None

In [25]:
from prompts import Prompts
prompts = Prompts()
prompt = prompts.single_meeting_2711

In [29]:
context_provider = MeetingContextProvider([meeting_id])

In [ ]:
async with get_session() as session:
    context = await context_provider.get_context(session=session)
context


In [ ]:
@jupyter_stream_output(markdown=True)
async def chat_with_meeting(thread_id, user_id, meeting_id, query):
    async with get_session() as session:
        context_provider = MeetingContextProvider([meeting_id])
        meeting_chat_manager = MeetingChatManager(session, context_provider=context_provider)
        async for response in meeting_chat_manager.chat(
            meeting_ids=[meeting_id],
            user_id=user_id,
            query=query,
            thread_id=thread_id,
            temperature=0.7,
            prompt=prompts.single_meeting_2711
        ):
            yield response
# Usage
response = await chat_with_meeting(
    thread_id=thread_id,
    user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
    meeting_id=meeting_id,
    query="summare meeting"
)

In [ ]:
meeting_

In [ ]:
await meeting_chat_manager.chat(user_id=user_id,query="What was the main topic of the meeting?",meeting_ids=[meeting_id])